In [10]:
import pandas as pd
import numpy as np
import re

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import spacy
import gensim


import plotly.express as px

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 300
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
#### title

In [14]:
# merging files
tweets_fr = pd.concat(
    map(pd.read_csv, ['../data/tweets_fr_week_2_translated.csv', '../data/tweets_fr_week4_translated.csv']), ignore_index=True)
tweets_fr.shape

(12558, 8)

In [4]:
tweets_us_north = pd.read_csv('../data/tweets_us_north.csv')
tweets_us_south = pd.read_csv('../data/tweets_us_south.csv')

In [ ]:
### 'title' use pretrained “distilbert-base-uncased-finetuned-sst-2-english” (default classifier from 🤗)

In [9]:
classifier = pipeline('sentiment-analysis')

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

In [24]:
test["score"] = test["translated_text"].apply(classifier)

/Users/armelleleguelte/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
test

,id,date,location,follower_count,retweets,text,vaccine,translated_text,score
0,1393495540940300288,2021-05-15 09:16:41,Bordeaux,1133,0,"Ah ca c'est un autre pb. De toute facon, la limitation, c'est le nombre de doses achetées (les graph sur VaccinTracker motre bien qu'a part Aztrazeneca, on consomme tout ce qui nous est livré). Donc tant que ca produira pas plus...",astrazeneca,"This is another pb. In any case, the limitation is the number of doses purchased (the graphs on VaccinTracker motor although from Aztrazeneca, we consume everything that is delivered to us). So as long as it does not produce more...","[{'label': 'NEGATIVE', 'score': 0.9971478581428528}]"
1,1393468639328747520,2021-05-15 07:29:47,France,93,0,100% des cas graves pour Aztrazeneca. 95/97 pour les autres.,astrazeneca,100% serious cases for Aztrazeneca. 95/97 for others.,"[{'label': 'NEGATIVE', 'score': 0.7635113596916199}]"


In [26]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               2 non-null      int64 
 1   date             2 non-null      object
 2   location         2 non-null      object
 3   follower_count   2 non-null      int64 
 4   retweets         2 non-null      int64 
 5   text             2 non-null      object
 6   vaccine          2 non-null      object
 7   translated_text  2 non-null      object
 8   score            2 non-null      object
dtypes: int64(3), object(6)
memory usage: 272.0+ bytes


In [ ]:
# Need to split on the "," to get 2 columns then can split "'" to get Neg and number